# Importing some Important library

In [1]:
import numpy as np
import pandas as pd

import re
from tqdm import tqdm

import collections

from sklearn.cluster import KMeans

from nltk.stem import WordNetLemmatizer  # For Lemmetization of words
from nltk.corpus import stopwords  # Load list of stopwords
from nltk import word_tokenize # Convert paragraph in tokens

import pickle
import sys

from gensim.models import word2vec # For represent words in vectors
import gensim

# Reading Provided Data-Set

In [16]:
# Read given data-set using pandas
text_data = pd.read_csv("Precily_Text_Similarity.csv")
print("Shape of text_data : ", text_data.shape)
text_data.head()

Shape of text_data :  (3000, 3)


,Unique_ID,text1,text2
0,0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [3]:
#Checking whether is it having NAN values or not.
text_data.isna().values.any() 

False

In [4]:
text_data.isnull().sum() # Check if text data have any null values

text1    0
text2    0
dtype: int64

# Preprocessing of text1 & text2

In [5]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [6]:
# Combining all the above stundents 

preprocessed_text1 = []

# tqdm is for printing the status bar

for sentance in tqdm(text_data['text1'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text1.append(sent.lower().strip())

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [07:08<00:00,  7.00it/s]


In [7]:
# Merging preprocessed_text1 in text_data
text_data['text1'] = preprocessed_text1
text_data.head(3)

,text1,text2
0,broadband challenges tv viewing number europea...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure the lack of ...
2,player burn worries robinson england coach and...,hanks greeted at wintry premiere hollywood sta...


In [8]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_text2 = []

# tqdm is for printing the status bar
for sentance in tqdm(text_data['text2'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
   
    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text2.append(sent.lower().strip())

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [07:20<00:00,  6.82it/s]


In [9]:
# Merging preprocessed_text2 in text_data

text_data['text2'] = preprocessed_text2

text_data.head(3)

,text1,text2
0,broadband challenges tv viewing number europea...,gardener wins double glasgow britain jason gar...
1,rap boss arrested drug find rap mogul marion s...,amnesty chief laments war failure lack public ...
2,player burn worries robinson england coach and...,hanks greeted wintry premiere hollywood star t...


In [10]:
def word_tokenizer(text):
            #tokenizes and stems the text
            tokens = word_tokenize(text)
            lemmatizer = WordNetLemmatizer() 
            tokens = [lemmatizer.lemmatize(t) for t in tokens]
            return tokens

# Proposed Approach

# Word embeddings :

Word embeddings are low dimensional vectors obtained by training a neural network on a large corpus to predict a word given a context (Continuous Bag Of Words model) or to predict the context given a word (skip gram model). The context is a window of surrounding words. Pre-trained word embeddings are also available in the word2vec code.google page.

In this i am using Google news pre trained vectors and compare similarity between text1 & text2 using n_similarity method in gensim library which is nothing compares cosine similarity between two

Considering it as a unsupervised problem.

In [11]:
# Loading pre_trained Google News Vectors after download file
wordmodelfile="GoogleNews-vectors-negative300.bin"
wordmodel= gensim.models.KeyedVectors.load_word2vec_format(wordmodelfile, binary=True)

In [12]:
# This code check if word in text1 & text2 present in our google news vectors vocabalry.
# if not it removes that word and if present it compares similarity score between text1 and text2 words

similarity = [] # List for store similarity score

for ind in text_data.index:
    
        s1 = text_data['text1'][ind]
        s2 = text_data['text2'][ind]
        
        if s1==s2:
                 similarity.append(0.0) # 0 means highly similar
                
        else:   

            s1words = word_tokenizer(s1)
            s2words = word_tokenizer(s2)        
            
            vocab = wordmodel.key_to_index #the vocabulary considered in the word embeddings
            
            if len(s1words and s2words)==0:
                    similarity.append(1.0)

            else:
                
                for word in s1words.copy(): #remove sentence words not found in the vocab
                    if (word not in vocab):
                        
                            s1words.remove(word)
                                           
                for word in s2words.copy(): #idem

                    if (word not in vocab):
                           
                            s2words.remove(word)
                                                       
                similarity.append((1-wordmodel.n_similarity(s1words, s2words))) # as it is given 1 means highly dissimilar & 0 means highly similar

# Final Submission

Make Final DataFrame and save a CSV file of similarity scores with Unique_ID. (Columns : Unique_ID, Similarity_Score)

In [17]:
# Get Unique_ID and similarity
final_score = pd.DataFrame({'Unique_ID':text_data.Unique_ID,'Similarity_score':similarity})
final_score.head(3) 

,Unique_ID,Similarity_score
0,0,0.314564
1,1,0.366671
2,2,0.282510


In [18]:
# SAVE DF as CSV file 
final_score.to_csv('final_score.csv',index=False)

# THANKS !!!   END OF ASSIGNMENT 